In [341]:
using LinearAlgebra, SparseArrays
import gmsh
gmsh.initialize()

include("../src/FEM.jl")
using .FEM

LoadError: LoadError: ParseError:
# Error @ /home/perebal/Dokumentumok/GitHub/Oktatas/src/FEM.jl:133:101
        elementName, dim, order, numNodes::Int64, localNodeCoord, numPrimaryNodes = gmsh.model.mesh.getElementProperties(et)
        intPoints, intWeights = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order+1)))
#                                                                                                   └ ── Expected `end`
in expression starting at /home/perebal/Dokumentumok/GitHub/Oktatas/src/FEM.jl:133

In [342]:
gmsh.open("cantilever1D.geo")

Info    : Reading 'cantilever1D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000192943s, CPU 0.000189s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Blossom: 3 internal 4 closed
Info    : Blossom recombination completed (Wall 4.8122e-05s, CPU 5e-05s): 2 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 1, min Q = 1
Info    : Done meshing 2D (Wall 0.000314141s, CPU 0.000305s)
Info    : 6 nodes 12 elements
Info    : Done reading 'cantilever1D.geo'


In [343]:
problem = FEM.Problem(type="PlaneStress", E=2.0e5, ν=0.0, ρ=1.0)

Main.FEM.Problem("cantilever", "PlaneStress", 2, 200000.0, 0.0, 1.0, 1.0, 6)

In [344]:
#gmsh.fltk.run()

In [345]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [346]:
K = FEM.stiffnessMatrix(problem)

12×12 SparseMatrixCSC{Float64, Int64} with 112 stored entries:
      1.81481e5   25000.0        …   -88518.5           -25000.0
  25000.0             3.40741e5      -25000.0               -1.69259e5
       ⋅               ⋅             -71481.5            25000.0
       ⋅               ⋅              25000.0          -132963.0
       ⋅               ⋅              51481.5           -25000.0
       ⋅               ⋅         …    25000.0           122963.0
     -1.61481e5  -25000.0             68518.5            25000.0
  25000.0            -3.30741e5      -25000.0                1.59259e5
  68518.5         25000.0           -340000.0                0.0
 -25000.0             1.59259e5           3.63798e-12       -6.97778e5
 -88518.5        -25000.0        …   380000.0                0.0
 -25000.0            -1.69259e5           0.0                7.17778e5

In [347]:
#M = FEM.massMatrix(problem, lumped=false)

In [348]:
#dof, dof = size(M)
#a = zeros(dof)
#a[1:2:dof] .= 10
#fx = M * a

In [349]:
supp1 = FEM.displacementConstraint("bottom", uy=0)
supp2 = FEM.displacementConstraint("left", ux=0)
load1 = FEM.load("right", fx=10)
#load2 = FEM.traction("bottom", fx=1)
#load3 = FEM.traction("top", fx=1)
load4 = FEM.load("body", fx=1)

("body", 1, 0, 0)

In [350]:
f = FEM.loadVector(problem, [load1 load4])

12-element Vector{Float64}:
 125.0
   0.0
 175.0
   0.0
 175.0
   0.0
 125.0
   0.0
 250.0
   0.0
 250.0
   0.0

In [351]:
K1, f1 = FEM.applyBoundaryConditions(problem, K, f, [supp1 supp2])

(sparse([1, 2, 3, 5, 6, 9, 11, 12, 4, 3  …  5, 6, 8, 9, 11, 3, 5, 6, 8, 12], [1, 2, 3, 3, 3, 3, 3, 3, 4, 5  …  11, 11, 11, 11, 11, 12, 12, 12, 12, 12], [1.0, 1.0, 198518.51851851857, -178518.51851851854, -25000.000000000007, 51481.48148148148, -71481.48148148149, 25000.000000000007, 1.0, -178518.51851851854  …  51481.48148148148, 25000.000000000007, -25000.000000000007, -340000.0000000001, 380000.0000000001, 25000.000000000004, -25000.000000000004, 122962.96296296298, 159259.25925925927, 717777.777777778], 12, 12), [0.0, 0.0, 175.0, 0.0, 175.0, 0.0, 0.0, 0.0, 250.0, 0.0, 250.0, 0.0])

In [352]:
q = FEM.solveDisplacement(K1, f1)

12-element Vector{Float64}:
  0.0
  0.0
  0.029999999999999968
  0.0
  0.029999999999999975
  0.0
  0.0
 -1.750043724066276e-19
  0.02124999999999999
  0.0
  0.02124999999999999
  2.205710509392185e-19

In [353]:
FT = K * q - f

12-element Vector{Float64}:
 -550.0
    7.635314502560757e-14
   -9.094947017729282e-13
    8.435905724488289e-14
    9.094947017729282e-13
    1.4080890768895697e-13
 -550.0
   -7.855885553499974e-14
    0.0
   -2.379753050474156e-13
    9.094947017729282e-13
    1.6763094347034258e-14

In [354]:
S = FEM.solveStress(problem, q)

Main.FEM.StressField([[34.99999999999992; 4.336808689942018e-14; … ; 0.0; 0.0;;], [84.99999999999996; 0.0; … ; 0.0; 0.0;;]], [7, 8], 1)

In [355]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [356]:
#u = FEM.showDoFResults(problem, q, "uvec", name="uvec", visible=false)
ux = FEM.showDoFResults(problem, q, "ux", name="ux", visible=false)
#uy = FEM.showDoFResults(problem, q, "uy", name="uy", visible=false)
#uz = FEM.showDoFResults(problem, q, "uz", name="uz", visible=false)
#s = FEM.showStressResults(problem, S, "s", name="σ", visible=true, smooth=true)
sx = FEM.showStressResults(problem, S, "sx", name="σx", visible=false, smooth=false)
gmsh.view.option.setNumber(sx, "MaxRecursionLevel", 0)
#sy = FEM.showStressResults(problem, S, "sy", name="σy", visible=false, smooth=true)
#sz = FEM.showStressResults(problem, S, "sz", name="σz", visible=false, smooth=true)
#sxy = FEM.showStressResults(problem, S, "sxy", name="τxy", visible=false, smooth=true)
#syz = FEM.showStressResults(problem, S, "syz", name="τyz", visible=false, smooth=true)
#szx = FEM.showStressResults(problem, S, "szx", name="τzx", visible=false, smooth=true)
ft = FEM.showDoFResults(problem, FT, "uvec", name="F_T", visible=false)
gmsh.view.option.setNumber(ft, "MaxRecursionLevel", 0)

"ux..ok"

"sx..ok"

"uvec..ok"

In [357]:
FEM.plotOnPath(problem, "path", sx, 1000, name="σx", visible=true);
#FEM.plotOnPath(problem, "path", sxy, 100, name="τxy", visible=false);
FEM.plotOnPath(problem, "path", ux, 1000, name="ux", visible=true);

In [358]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [359]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.2
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20240121
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [360]:
gmsh.finalize()